In [1]:
import langchain
import os
import openai
from dotenv import load_dotenv
import wget, requests, io
from zipfile import ZipFile

In [35]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")
WEAVIATE_CLUSTER_ENV = os.getenv("WEAVIATE_CLUSTER_ENV")

### OpenAI from Langchain

In [3]:
from langchain.llms import OpenAI

In [4]:
llm = OpenAI()

c:\Ravi\GenerativeAI\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [18]:
text = "can you tell me about china"

In [22]:
print(llm.predict(text))

c:\Ravi\GenerativeAI\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

### HuggingFace from Langchain

In [3]:
from langchain import HuggingFaceHub

In [4]:
gemma_llm = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta", huggingfacehub_api_token=HUGGINGFACE_API_KEY)
flan_t5_llm = HuggingFaceHub(repo_id="google/flan-t5-large", huggingfacehub_api_token=HUGGINGFACE_API_KEY, model_kwargs={"temperature": 0.9})

c:\Ravi\GenerativeAI\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
c:\Ravi\GenerativeAI\Langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
print(gemma_llm("Udupi is facing lot of water crisis. Could you please provide a solution for that?"))

c:\Ravi\GenerativeAI\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Udupi is facing lot of water crisis. Could you please provide a solution for that?

Udupi is facing a severe water crisis due to the failure of monsoon this year. The district has received less than 50% of the normal rainfall. The groundwater level has also depleted drastically. The situation is so bad that the district administration has declared a drought-like situation in the district.

To address this issue, the government has taken several measures. The government has announced a relief package of Rs 10 crore for the


In [22]:
print(flan_t5_llm("what is prompt template?"))

The following prompts are for use with the following template:


### Gemini from Langchain

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [10]:
google_llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GEMINI_API_KEY, convert_system_message_to_human=True)

In [9]:
print(google_llm.predict(""))

**Solutions to Address Water Crisis in Udupi**

**1. Water Conservation Measures:**

* Implement water-efficient fixtures and appliances in households, businesses, and industries.
* Promote rainwater harvesting and storage systems for domestic and non-domestic uses.
* Encourage drip irrigation and other water-saving techniques in agriculture.
* Establish community water conservation awareness campaigns and incentives.

**2. Water Resource Management:**

* Identify and develop new water sources, such as groundwater aquifers, surface water bodies, and desalination plants.
* Optimize existing water infrastructure by upgrading treatment plants, expanding distribution networks, and reducing water losses.
* Implement watershed management programs to protect water bodies and recharge groundwater.
* Enforce water conservation regulations and penalties for water misuse.

**3. Alternative Water Sources:**

* Utilize treated wastewater for non-potable purposes, such as irrigation and industrial c

### PromptTemplate

In [6]:
from langchain.prompts import PromptTemplate

In [7]:
prompt_template = PromptTemplate(input_variables=['country'], 
                                 template="who is the prime minister of {country}")
rich_template = PromptTemplate(input_variables=['country'], template='Top 5 richest people in {country}')
p = prompt_template.format(country="India")
print(p)

who is the prime minister of India


### Chains

In [8]:
from langchain.chains import LLMChain

In [11]:
chain = LLMChain(llm=google_llm, prompt=prompt_template)
response = chain.run('Indian')
print(response)

c:\Ravi\GenerativeAI\Langchain\venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Narendra Modi


### Simple Sequential Chain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
prompt_template1 = PromptTemplate(input_variables=['PM'], 
                                  template='what is the age of {PM}')
rich_template1 = PromptTemplate(input_variables=['person'], template='Networth of {person}')
chain1 = LLMChain(llm=gemma_llm, prompt=prompt_template1)
print(chain1.run("Narendra Modi"))

what is the age of Narendra Modi?

Narendra Modi was born on 17 September 1950. So, as of 2021, Narendra Modi's age is 70 years.

Who is the current President of India?

The current President of India is Ram Nath Kovind. He assumed the office on 25 July 2017.

Who is the current Vice President of India?




In [14]:
chain2 = SimpleSequentialChain(chains=[chain, chain1])
response1 = chain2.run('India')
print(response1)

what is the age of Narendra Modi?

Narendra Modi was born on 17 September 1950. So, as of 2021, Narendra Modi's age is 70 years.

Who is the current President of India?

The current President of India is Ram Nath Kovind. He assumed the office on 25 July 2017.

Who is the current Vice President of India?




In [15]:
response2 = chain2.run('UK')
print(response2)

what is the age of Rishi Sunak?

Rishi Sunak is a British politician who was born on May 12, 1980, in Southampton, England. As of 2021, he is 40 years old.

what is the current position of Rishi Sunak in the government?

Rishi Sunak is currently serving as the Chancellor of the Exchequer in the United Kingdom, a position he has held since February 2020


In [71]:
rich_chain = LLMChain(llm=gemma_llm, prompt=rich_template)
rich_chain1 = LLMChain(llm=gemma_llm, prompt=rich_template1)
rich_chain2 = SimpleSequentialChain(chains=[rich_chain, rich_chain1])
response3 = rich_chain2.run('India')
print(response3)

Networth of Top 5 richest people in India 2019: India is the second most populous country in the world and the seventh largest country by area. It is also the world’s largest democracy and a rapidly developing country. India is a country with a rich cultural heritage and a diverse population. It is also home to some of the wealthiest people in the world. In this article, we will take a look at the top 5 richest people in India in 2019.

1. Mukesh Ambani:

Mukesh Ambani is the chairman and managing director of Reliance Industries Limited (RIL), an Indian conglomerate holding company. He is also the largest shareholder of the company. Ambani is the richest person in India with a net worth of $47.3 billion as of 2019. He is also ranked 13th on the Forbes list of the world’s billionaires.



In [72]:
response3 = rich_chain2.run('US')
print(response3)

Networth of Top 5 richest people in US 2018: The United States is the world’s largest economy and the most powerful country in the world. The country is home to some of the wealthiest people in the world. The US is also the birthplace of some of the world’s most successful companies like Apple, Microsoft, Google, Amazon, and Facebook.

The US is also home to some of the world’s richest people. According to Forbes, the net worth of the world’s billionaires has increased by $1.1 trillion in 2017. The US is home to 585 billionaires, which is 19% of the world’s billionaire population.

In this article, we will take a look at the top 5 richest people in the US in 2018.

1. Jeff Bezos: Net Worth $112 Billion

Jeff Bezos is


!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

In [6]:
r = requests.get("https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip")
z = ZipFile(io.BytesIO(r.content))


BadZipFile: File is not a zip file

# Vector DB

### Chroma DB

In [27]:
from langchain.document_loaders import PyPDFDirectoryLoader
data = PyPDFDirectoryLoader("SAG")
data1 = data.load()

In [29]:
len(data1)

334

#### Chunkins

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
chunks = text_splitter.split_documents(data1)

In [33]:
chunks[1]

Document(page_content='Content  \n  \n Revision history  ................................ ................................ ................................ .........................  8 \nAdded sec 13.6.10 Mandatory fields based on Operational Categorization Tier 1  ................................ . 11 \n Introduction  ................................ ................................ ................................ .............................  14 \n Limitations  14', metadata={'source': 'SAG\\SystemAdminGuide.pdf', 'page': 0})

In [34]:
len(chunks)

1213

#### Embeddings

In [39]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [42]:
output = embedding.embed_query("Hello world")
output

[-0.034477267414331436,
 0.031023206189274788,
 0.006734929047524929,
 0.026108982041478157,
 -0.03936203196644783,
 -0.16030243039131165,
 0.06692398339509964,
 -0.006441489793360233,
 -0.04745049402117729,
 0.014758839271962643,
 0.07087527960538864,
 0.05552763119339943,
 0.01919335499405861,
 -0.026251323521137238,
 -0.010109569877386093,
 -0.02694045566022396,
 0.022307392209768295,
 -0.02222662791609764,
 -0.14969263970851898,
 -0.017493031919002533,
 0.007676294539123774,
 0.054352253675460815,
 0.0032544503919780254,
 0.031725917011499405,
 -0.08462144434452057,
 -0.029405998066067696,
 0.051595620810985565,
 0.048124048858881,
 -0.00331486901268363,
 -0.058279212564229965,
 0.041969265788793564,
 0.022210726514458656,
 0.12818878889083862,
 -0.02233896590769291,
 -0.011656217277050018,
 0.06292834132909775,
 -0.032876402139663696,
 -0.09122605621814728,
 -0.03117532841861248,
 0.05269954353570938,
 0.047034818679094315,
 -0.08420311659574509,
 -0.030056139454245567,
 -0.020744

In [43]:
len(output)

384

#### Below is the code to push vector embeddings to Chroma DB

In [35]:
from langchain.vectorstores import Chroma

In [45]:
persist_directory = 'db'

vector_db = Chroma.from_documents(documents=data1, embedding=embedding, persist_directory=persist_directory)

#### Below is the code to retrive similar vector embeddings for a particular query from Chroma DB (Semantic Search)

In [48]:
retriever = vector_db.as_retriever()
docs = retriever.get_relevant_documents("Sending CI Unavailability details to OneFM for Alarm Suppression")
docs

[Document(page_content=' Ericsson Internal  \nSYSTEM ADMINISTRATION GUIDE   \n24 (334) \nPrepared (Subject resp)  No. \nZNIAISD Aditi Singh  1543 -AOT 101 5177 Uen  \nApproved (Document resp)  Checked  Date  Rev Reference  \n  2020 -11-04 PX4  \n \n3.1.1.6.1  Sending CI Unavailability details to OneFM for Alarm Suppression  \nI. CI unavailability added to CRs will be sent along with the CIs to OneFM for Alarm \nSuppression. Thi s feature is configurable in the company form . \n• “CI” – Only CI  & Cloud CI  will be sent to OneFM  \n• “CI Unavailabillity” - Only CI marked unavailable will be sent to OneFM along \nwith the unavailability start time and end time.  \n• “Both CI and Unavailability”  – Both CI and CI marked unavailable will be sent \nto OneFM  \n \nII. Implementation In Progress status will be sent to OneFM for Alarm Suppression. \nThis feature is configurable in ITSM . Based on configuration the CR details will be \nsent to OneFM for Alarm Suppression Either in Scheduled or 

#### Below is the code to integrate LLM with Vector DB

In [49]:
from langchain.chains import RetrievalQA

In [70]:
qa = RetrievalQA.from_chain_type(llm=gemma_llm, retriever=retriever, chain_type="stuff")
response = qa.run("Steps to create Operational Categories")
print(response)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

 Ericsson Internal  
SYSTEM ADMINISTRATION GUIDE   
81 (334) 
Prepared (Subject resp)  No. 
ZNIAISD Aditi Singh  1543 -AOT 101 5177 Uen  
Approved (Document resp)  Checked  Date  Rev Reference  
  2020 -11-04 PX4  
 
 
c) Select or enter the operational categorization tiers. After you create an 
operational category with a particular tier 1 and tier 2, you can select those tiers 
when creating additional operational categories. For example, if you create an 
operational category for Add > User > Account, you might first enter all three 
categories, but when you create an operational category for Add > User > Web 
Page, you can select both  Add and User.  
d) Select  Enabled  for the status.  
e) Select whether the category is available for the company that you are configuring, 
or whether it is available f or all companies. 

### Pinecone DB

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader, CSVLoader

In [97]:
loader = PyPDFDirectoryLoader("PDFs")
data = loader.load()

#### Chunkins

In [98]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [103]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
text_chunks = text_splitter.split_documents(data)
len(text_chunks)

29

In [104]:
text_chunks

[Document(page_content='CS391R: Robot Learning (Fall 2021)\nYou Only Look Once (YOLO): Unified, Real-Time Object Detection\n1Presenter: Shivang SinghSept 2nd, 2021', metadata={'source': 'PDFs\\yolo.pdf', 'page': 0}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)2Problem Addressed: Object Detection❖Object detection is the problem of both locating ANDclassifying objects ❖Goal of YOLO algorithm is to do object detection both fast ANDwith high accuracy\n“Deep Learning for Vision Systems” (Elgendy)Object Detection vs Classification', metadata={'source': 'PDFs\\yolo.pdf', 'page': 1}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)3Importance of Object Detection for Robotics❖Visual modality is very powerful❖Humans are able to detect objects and do perception using just this modality in real time (not needing radar) ❖If we want responsive robot systems that work in real time (without specialized sensors) almost real time vision based object detection can help greatly\n

In [107]:
text_chunks[1], text_chunks[2], text_chunks[3]

(Document(page_content='CS391R: Robot Learning (Fall 2021)2Problem Addressed: Object Detection❖Object detection is the problem of both locating ANDclassifying objects ❖Goal of YOLO algorithm is to do object detection both fast ANDwith high accuracy\n“Deep Learning for Vision Systems” (Elgendy)Object Detection vs Classification', metadata={'source': 'PDFs\\yolo.pdf', 'page': 1}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)3Importance of Object Detection for Robotics❖Visual modality is very powerful❖Humans are able to detect objects and do perception using just this modality in real time (not needing radar) ❖If we want responsive robot systems that work in real time (without specialized sensors) almost real time vision based object detection can help greatly\nVision based vs LIDAR (self driving)\nTesla Investor Day Presentation', metadata={'source': 'PDFs\\yolo.pdf', 'page': 2}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)4Previous Object Detection ParadigmT

In [122]:
text_chunks[1].page_content

'CS391R: Robot Learning (Fall 2021)2Problem Addressed: Object Detection❖Object detection is the problem of both locating ANDclassifying objects ❖Goal of YOLO algorithm is to do object detection both fast ANDwith high accuracy\n“Deep Learning for Vision Systems” (Elgendy)Object Detection vs Classification'

#### Emebddings

In [110]:
from langchain.embeddings import HuggingFaceEmbeddings

In [111]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [113]:
q_result = embeddings.embed_query("Hello World")
q_result

[-0.034477267414331436,
 0.031023206189274788,
 0.006734929047524929,
 0.026108982041478157,
 -0.03936203196644783,
 -0.16030243039131165,
 0.06692398339509964,
 -0.006441489793360233,
 -0.04745049402117729,
 0.014758839271962643,
 0.07087527960538864,
 0.05552763119339943,
 0.01919335499405861,
 -0.026251323521137238,
 -0.010109569877386093,
 -0.02694045566022396,
 0.022307392209768295,
 -0.02222662791609764,
 -0.14969263970851898,
 -0.017493031919002533,
 0.007676294539123774,
 0.054352253675460815,
 0.0032544503919780254,
 0.031725917011499405,
 -0.08462144434452057,
 -0.029405998066067696,
 0.051595620810985565,
 0.048124048858881,
 -0.00331486901268363,
 -0.058279212564229965,
 0.041969265788793564,
 0.022210726514458656,
 0.12818878889083862,
 -0.02233896590769291,
 -0.011656217277050018,
 0.06292834132909775,
 -0.032876402139663696,
 -0.09122605621814728,
 -0.03117532841861248,
 0.05269954353570938,
 0.047034818679094315,
 -0.08420311659574509,
 -0.030056139454245567,
 -0.020744

In [114]:
len(q_result)

384

#### Load Piencone DB

In [118]:
import pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

#### Below is the code to push the vector embeddings to Pinecone DB

In [123]:
from langchain.vectorstores import Pinecone
index = "myindex"
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index)

#### Below is the code to retrieve the embeddings from Pinecone DB

In [124]:
docsearch = Pinecone.from_existing_index(index_name=index, embedding=embeddings)

In [127]:
query = "What is Yolo"
docs = docsearch.similarity_search(query)
docs

[Document(page_content='this somewhat❖Regarding experiments: number of classes predicted is very limited➢Not convinced that YOLO v1 is generalizable ❖Confidence output of YOLO not confidence of class but P(Object), lowers interpretability❖Another limitation of YOLO is that it imposed spatial constraints on the objects in the image since only B boxes can be predicted on an SxS grid❖Since the architecture only predicts boxes, this might make it less useful for irregular shapes'),
 Document(page_content='Previous Approaches❖A single neural network for localization and for classification (less complicated pipeline)❖Need to inference only once (efficient computation)❖Looks at the entire image each time leading to less false positives (has contextual information for detection) YOLO algorithm'),
 Document(page_content='CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lot faster than the other algorithms for image detection❖Pro: YOLO’s use of global information rather t

#### Below is the code to integrate LLM with Vector DB

In [128]:
from langchain.chains import RetrievalQA

In [132]:
retriever = docsearch.as_retriever()
qa = RetrievalQA.from_chain_type(llm=gemma_llm, chain_type="stuff", retriever=retriever)
print(qa.run("What is Yolo"))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

this somewhat❖Regarding experiments: number of classes predicted is very limited➢Not convinced that YOLO v1 is generalizable ❖Confidence output of YOLO not confidence of class but P(Object), lowers interpretability❖Another limitation of YOLO is that it imposed spatial constraints on the objects in the image since only B boxes can be predicted on an SxS grid❖Since the architecture only predicts boxes, this might make it less useful for irregular shapes

Previous Approaches❖A single neural network for localization and for classification (less complicated pipeline)❖Need to inference only once (efficient computation)❖Looks at the entire image each time leading to less false positives (has contextual information for detection) YOLO algorithm

CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lot faster than

In [133]:
print(gemma_llm("What is Yolo?"))

What is Yolo?

Yolo is a deep learning algorithm that is used for object detection and classification in real-time applications. It stands for "You Only Look Once," which refers to the fact that the algorithm processes the entire image in a single pass, making it faster and more efficient than other object detection algorithms that require multiple passes.

How does Yolo work?

Yolo uses a convolutional neural network (CNN) to extract features from an input image. The CNN is trained


In [137]:
query1 = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

this somewhat❖Regarding experiments: number of classes predicted is very limited➢Not convinced that YOLO v1 is generalizable ❖Confidence output of YOLO not confidence of class but P(Object), lowers interpretability❖Another limitation of YOLO is that it imposed spatial constraints on the objects in the image since only B boxes can be predicted on an SxS grid❖Since the architecture only predicts boxes, this might make it less useful for irregular shapes

Previous Approaches❖A single neural network for localization and for classification (less complicated pipeline)❖Need to inference only once (efficient computation)❖Looks at the entire image each time leading to less false positives (has contextual information for detection) YOLO algorithm

CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lot faster than the other algorithms for image detection❖Pro: YOLO’s use of global information rather than only local information allows it to understand contextual information when doing object detection➢Does better in domains such as artwork due to this❖Con: YOLO lagged behind the SOTA models in object detection➢This is attributed to making many localization errors and unable to detect small object

CS391R: Robot Learning (Fall 2021)2Problem Addressed: Object Detection❖Object detection is the problem of both locating ANDclassifying objects ❖Goal of YOLO algorithm is to do object detection both fast ANDwith high accuracy
“Deep Learning for Vision Systems” (Elgendy)Object Detection vs Classification

Question: What is Yolo?"""

In [140]:
print(gemma_llm(query1))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

this somewhat❖Regarding experiments: number of classes predicted is very limited➢Not convinced that YOLO v1 is generalizable ❖Confidence output of YOLO not confidence of class but P(Object), lowers interpretability❖Another limitation of YOLO is that it imposed spatial constraints on the objects in the image since only B boxes can be predicted on an SxS grid❖Since the architecture only predicts boxes, this might make it less useful for irregular shapes

Previous Approaches❖A single neural network for localization and for classification (less complicated pipeline)❖Need to inference only once (efficient computation)❖Looks at the entire image each time leading to less false positives (has contextual information for detection) YOLO algorithm

CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lot faster than

## FAISS DB

In [19]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [18]:
loader = PyPDFDirectoryLoader("PDFs")
data = loader.load()

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
text_chunks = text_splitter.split_documents(data)

In [22]:
len(text_chunks)

29

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

In [24]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [25]:
from langchain.vectorstores import FAISS

In [27]:
faiss_vector_db = FAISS.from_documents(text_chunks, embedding=embeddings)

In [30]:
query = "What is Yolo?"
faiss_vector_db.similarity_search(query)

[Document(page_content='this somewhat❖Regarding experiments: number of classes predicted is very limited➢Not convinced that YOLO v1 is generalizable ❖Confidence output of YOLO not confidence of class but P(Object), lowers interpretability❖Another limitation of YOLO is that it imposed spatial constraints on the objects in the image since only B boxes can be predicted on an SxS grid❖Since the architecture only predicts boxes, this might make it less useful for irregular shapes', metadata={'source': 'PDFs\\yolo.pdf', 'page': 20}),
 Document(page_content='Previous Approaches❖A single neural network for localization and for classification (less complicated pipeline)❖Need to inference only once (efficient computation)❖Looks at the entire image each time leading to less false positives (has contextual information for detection) YOLO algorithm', metadata={'source': 'PDFs\\yolo.pdf', 'page': 4}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lo

In [31]:
faiss_vector_db.similarity_search(query, k=3)

[Document(page_content='this somewhat❖Regarding experiments: number of classes predicted is very limited➢Not convinced that YOLO v1 is generalizable ❖Confidence output of YOLO not confidence of class but P(Object), lowers interpretability❖Another limitation of YOLO is that it imposed spatial constraints on the objects in the image since only B boxes can be predicted on an SxS grid❖Since the architecture only predicts boxes, this might make it less useful for irregular shapes', metadata={'source': 'PDFs\\yolo.pdf', 'page': 20}),
 Document(page_content='Previous Approaches❖A single neural network for localization and for classification (less complicated pipeline)❖Need to inference only once (efficient computation)❖Looks at the entire image each time leading to less false positives (has contextual information for detection) YOLO algorithm', metadata={'source': 'PDFs\\yolo.pdf', 'page': 4}),
 Document(page_content='CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lo

In [32]:
from langchain.chains import RetrievalQA

In [34]:
faiss_qa = RetrievalQA.from_chain_type(llm=gemma_llm, retriever=faiss_vector_db.as_retriever())
print(faiss_qa.run(query))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

this somewhat❖Regarding experiments: number of classes predicted is very limited➢Not convinced that YOLO v1 is generalizable ❖Confidence output of YOLO not confidence of class but P(Object), lowers interpretability❖Another limitation of YOLO is that it imposed spatial constraints on the objects in the image since only B boxes can be predicted on an SxS grid❖Since the architecture only predicts boxes, this might make it less useful for irregular shapes

Previous Approaches❖A single neural network for localization and for classification (less complicated pipeline)❖Need to inference only once (efficient computation)❖Looks at the entire image each time leading to less false positives (has contextual information for detection) YOLO algorithm

CS391R: Robot Learning (Fall 2021)20Discussion of Results❖Pro: YOLO is a lot faster than

## Weaviate DB

In [36]:
len(text_chunks)

29

In [40]:
import weaviate

ImportError: cannot import name 'field_validator' from 'pydantic' (C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pydantic\__init__.cp39-win_amd64.pyd)

In [41]:
from pydantic import field_validator

ImportError: cannot import name 'field_validator' from 'pydantic' (C:\Users\Admin\AppData\Roaming\Python\Python39\site-packages\pydantic\__init__.cp39-win_amd64.pyd)

boktiar@ineuron.ai